<a href="https://colab.research.google.com/gist/aa19059/aabe021138505b45e9f8d792ac7cea16/stereo-depth-estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
pip install pypfm

In [ ]:
import numpy as np
from types import *
import torch 
from torchvision import transforms
import torch.nn as nn
import tensorflow as tf
from torch import optim
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from pypfm import PFMLoader
from google.colab.patches import cv2_imshow
import cv2
import os
import glob
from PIL import Image
from torch.utils.data import random_split

In [ ]:
class Depth(Dataset):


  def __init__(self, pathh, transform=None):
    self.loader = PFMLoader(color=False, compress=False)
    self.transform = transform
    self.pathh = pathh
    self.img_size = (320,240)
  
    self.files_left = sorted(glob.glob(os.path.join(self.pathh,'left','*.png')))
    self.files_right = sorted(glob.glob(os.path.join(self.pathh,'right','*.png')))
    self.pfm_array = sorted(glob.glob(os.path.join(self.pathh,'disparity','*.pfm')))
   

  def __len__(self):
    return len(self.pfm_array)
  
  

  def __getitem__(self, idx):
    
    image_left = Image.open(self.files_left[idx])
    image_left = image_left.resize(self.img_size)
    image_left = np.array(image_left)


    image_right = Image.open(self.files_right[idx])
    image_right= image_right.resize(self.img_size)
    image_right = np.array(image_right)

    pfm = self.loader.load_pfm(self.pfm_array[idx])
    pfm = np.array(pfm)
    pfm = cv2.resize(pfm,self.img_size)
    pfm = np.flip(pfm,axis=0).copy()
    target_tr = transforms.ToTensor()
    pfm = target_tr(pfm)

    if self.transform is not None:
        image_left = self.transform(image_left)
        image_right = self.transform(image_right)
    
    
    return image_left,  image_right, pfm
   
    
    

l = cv2.imread('/content/left/0006.png')

cv2_imshow(l)

p = cv2.imread('/content/right/0006.png')

cv2_imshow(p)

loader = PFMLoader(color=False, compress=False)

pfm= loader.load_pfm('/content/disparity/0006.pfm')

img = np.asarray(pfm).astype(np.uint8)
img = cv2.flip(img,0)

cv2_imshow(img)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
transform = transforms.Compose([
              transforms.ToTensor(),
              transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                    std=[0.229, 0.224, 0.225])
               ])

train_set = Depth('/content/drive/My Drive/Data/', transform)

x = len(train_set)

y = (80/100) * x

z = round(y)

a = x - round(y)

train_loader , train_validation = random_split(train_set,[z, a])

train_loader = DataLoader(train_loader,batch_size=32,shuffle=True,num_workers=8)

#print(len(train_loader))

train_validation = DataLoader(train_validation, batch_size=32, shuffle=True, num_workers=8)

#print(len(train_validation))


In [ ]:
model = nn.Sequential(nn.Conv2d(6,16,3,2,1),
                      nn.ReLU(),
                      nn.Conv2d(16,32,3,2,1),
                      nn.ReLU(),
                      nn.Conv2d(32,64,3,2,1),
                      nn.ReLU(),
                      nn.ConvTranspose2d(64,64,2,2),
                      nn.ReLU(),
                      nn.ConvTranspose2d(64,32,2,2),
                      nn.ReLU(),
                      nn.ConvTranspose2d(32,1,2,2),
                      nn.ReLU()
)
    

#model.cuda()



In [ ]:
input = torch.randn((1,6,240,320),dtype = torch.float32)

#model(input).size()

In [ ]:
criterion = nn.MSELoss() 
optimizer = optim.Adam(model.parameters(),lr=0.01, weight_decay=5e-4)
num_epochs = 50


In [ ]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    total = 0
    correct = 0

    for batch_idx, batch in enumerate (train_loader):
        img_l, img_r, target_pfm = batch
        img = torch.cat((img_l, img_r),1)
        outputs = model(img.cuda())
        optimizer.zero_grad()

        loss = criterion(outputs, target_pfm.cuda())
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()
    print('Results after epoch %d'% (epoch + 1))

    print('Training Loss: %.3f'%(train_loss/(batch_idx + 1)))
    
    model.eval()
    valid_loss = 0
    correct = 0
    total = 0

    for batch_idx, batch in enumerate (train_validation):
        img_l, img_r, target_pfm = batch
        img = torch.cat((img_l, img_r),1)
        with torch.no_grad():
            outputs = model(img.cuda())
            loss = criterion(outputs, target_pfm.cuda())

        valid_loss+= loss.item()
    print('Validation Loss: %.3f ' % (valid_loss / (batch_idx + 1)))
